```
Topic:        Project 2
Subject:      Predicting Premier League Players' Salaries
Date:         07/17/2020
Name:         David Weon

Notebook has been cleaned using nbextension 'Code prettify'
```

# Web Scraping Players' Stats in the Premier League

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import time, os
import requests

In [2]:
url = 'https://fbref.com/robots.txt'
response = requests.get(url)

In [3]:
url2 = 'https://www.spotrac.com/epl/manchester-united-f.c/payroll/2018/robots.txt'
response2 = requests.get(url2)

In [4]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# 2018-2019 Season Stats

## Manchester United 

### Standard Stats

In [5]:
def standardtable(teamURL):
    '''
    input: teamURL (str)
    output: dataframe of the standard stats table with selected stats
    '''
    #URL html for standard table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')

    #finding standard stats table
    standard_table = soup.find('table', id='stats_standard_ks_1889')

    #player names in list
    players = [
        header for header in standard_table.find_all('th', class_='left')
    ]
    player_names = [name.text for th in players for name in th.find_all('a')]

    #player stats in list
    player_row = [row for row in standard_table.find_all('tr')]

    player_stats = {}
    for player in player_row[2:]:
        items = player.find_all('td')
        player_stats[player] = [i.text for i in items]

    player_stats_list = [stats for stats in player_stats.values()]

    #matching player name with stats
    all_players_stats = {}
    all_players_stats.update(zip(player_names, player_stats_list))

    #creating dataframe for team's players and stats
    team_1819 = pd.DataFrame(all_players_stats).T
    team_1819.columns = [
        'Nation', 'Pos', 'Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'PK',
        'PKatt', 'CrdY', 'CrdR', 'P90Gls', 'P90Ast', 'P90G+A', 'P90G-PK',
        'P90G+A-PK', 'ExpectedxG', 'ExpectednpxG', 'ExpectedxA', 'P90xG',
        'P90xA', 'P90xG+xA', 'P90npxG', 'P90npxG+xA', 'Matches'
    ]

    #dropping unneded columns
    team_1819 = team_1819.drop([
        'Min', 'Nation', 'PK', 'PKatt', 'CrdR', 'Gls',
        'P90G+A', 'P90G-PK', 'P90G+A-PK', 'ExpectedxG', 'ExpectednpxG',
        'ExpectedxA', 'P90xG', 'P90xA', 'P90xG+xA', 'P90npxG', 'P90npxG+xA',
        'Matches', 'Ast'
    ],
                               axis=1)

    #data cleaning
    team_1819 = team_1819.replace(r'^\s*$', np.nan, regex=True)
    #team_1819['Min'] = team_1819['Min'].str.replace(',', '')
    team_1819.iloc[:, 1:] = team_1819.iloc[:, 1:].astype('float64')

    return team_1819

In [6]:
standardtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast
David de Gea,GK,27,38,38,1,0,0
Paul Pogba,MF,25,35,34,6,0.39,0.27
Victor Lindelöf,DF,24,30,29,1,0.03,0.03
Luke Shaw,DF,23,29,29,11,0.03,0.14
Ashley Young,DF,33,30,28,9,0.07,0.07
Nemanja Matić,MF,29,28,28,7,0.04,0
Marcus Rashford,FW,20,33,26,3,0.39,0.23
Chris Smalling,DF,28,24,24,1,0.04,0
Romelu Lukaku,FW,25,32,22,4,0.51,0
Jesse Lingard,"FW,MF",25,27,19,3,0.22,0.11


### Shooting Stats

In [7]:
from bs4 import BeautifulSoup, Comment

In [8]:
def shootingtable(teamURL):
    '''
    input: teamURL (str)
    output: dataframe of the shooting stats table with selected stats
    '''
    #URL html for shooting table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')
    placeholder = soup.select_one('#all_kitchen_sink_shooting .placeholder')
    comment = next(elem for elem in placeholder.next_siblings
                   if isinstance(elem, Comment))
    shootsoup = BeautifulSoup(comment, 'lxml')

    #finding shooting stats table
    shooting_table = shootsoup.find('table', id="stats_shooting_ks_1889")

    #player names in list
    shooters = [
        header for header in shooting_table.find_all('th', class_='left')
    ]
    shooter_names = [name.text for th in shooters for name in th.find_all('a')]

    #player stats in list
    shooting_row = [row for row in shooting_table.find_all('tr')]

    shooting_stats = {}
    for shooter in shooting_row[2:]:
        items = shooter.find_all('td')
        shooting_stats[shooter] = [i.text for i in items]
    shooting_stats_list = [stats for stats in shooting_stats.values()]

    #matching player name with stats
    all_shooting = {}
    all_shooting.update(zip(shooter_names, shooting_stats_list))

    #creating dataframe for team's players and stats
    shooting_df = pd.DataFrame(all_shooting).T
    shooting_df.columns = [
        'Nation', 'Pos', 'Age', '90s', 'Gls', 'PK', 'Pkatt', 'Shot',
        'ShotonTarget', 'FK', 'ShotonTarget%', 'Sh/90', 'SoT/90', 'G/Sh',
        'G/SoT', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches'
    ]

    #dropping unneded columns
    shooting_df = shooting_df.drop([
        'Nation', 'Pos', 'Age', 'Gls', 'PK', 'Pkatt', 'ShotonTarget',
        'G/Sh', 'G/SoT', 'xG', 'npxG', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches',
        'Shot', 'ShotonTarget%', 'FK'
    ],
                                   axis=1)

    #data cleaning
    shooting_df = shooting_df.replace(r'^\s*$', np.nan, regex=True)
    shooting_df.iloc[:, :] = shooting_df.iloc[:, :].astype('float64')

    return shooting_df

In [9]:
shootingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,90s,Sh/90,SoT/90
David de Gea,38.0,0.00,0.00
Paul Pogba,33.4,2.87,1.14
Victor Lindelöf,28.9,0.24,0.03
Luke Shaw,28.8,0.69,0.21
Ashley Young,28.5,0.42,0.11
Nemanja Matić,27.1,0.44,0.11
Marcus Rashford,25.9,3.20,1.54
Chris Smalling,23.6,0.51,0.17
Romelu Lukaku,23.7,2.49,1.39
Jesse Lingard,18.5,1.68,0.65


### Passing Stats

In [10]:
def passingtable(teamURL):
    '''
    input: teamURL(str)
    output: dataframe of the passing stats table with selected stats
    '''
    #URL html for shooting table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')
    placeholder = soup.select_one('#all_kitchen_sink_passing .placeholder')
    comment = next(elem for elem in placeholder.next_siblings
                   if isinstance(elem, Comment))
    passsoup = BeautifulSoup(comment, 'lxml')

    #finding shooting stats table
    passing_table = passsoup.find('table', id='stats_passing_ks_1889')

    #player names in list
    passers = [
        header for header in passing_table.find_all('th', class_='left')
    ]
    passer_names = [name.text for th in passers for name in th.find_all('a')]

    #player stats in list
    passing_row = [row for row in passing_table.find_all('tr')]

    passing_stats = {}
    for passer in passing_row[2:]:
        items = passer.find_all('td')
        passing_stats[passer] = [i.text for i in items]
    passing_stats_list = [stats for stats in passing_stats.values()]

    #matching player name with stats
    all_passing = {}
    all_passing.update(zip(passer_names, passing_stats_list))

    #creating dataframe for team's players and stats
    passing_df = pd.DataFrame(all_passing).T
    passing_df.columns = [
        'Nation', 'Pos', 'Age', '90s', 'TotPassCmp', 'TotPassAtt',
        'TotPassCmpPerc', 'TotDist', 'TotPrgDist', 'ShortCmp', 'ShortAtt',
        'ShortCmp%', 'MediumCmp', 'MediumAtt', 'MediumCmp%', 'LongCmp',
        'LongAtt', 'LongCmp%', 'Ast', 'xA', 'A-xA', 'KP', '1/3', 'PPA',
        'CrsPA', 'Prog', 'Matches'
    ]

    #dropping unneded columns
    passing_df = passing_df.drop([
        'Nation', 'Pos', 'Age', '90s', 'TotDist', 'ShortCmp', 'ShortAtt',
        'ShortCmp%', 'MediumCmp', 'MediumAtt', 'MediumCmp%', 'LongCmp',
        'LongAtt', 'LongCmp%', 'Ast', 'xA', 'A-xA', '1/3', 'PPA',
        'CrsPA', 'TotPrgDist', 'Matches', 'TotPassAtt', 'TotPassCmpPerc'
    ],
                                 axis=1)

    #data cleaning
    passing_df = passing_df.replace(r'^\s*$', np.nan, regex=True)
    passing_df.iloc[:, :] = passing_df.iloc[:, :].astype('float64')

    return passing_df

In [11]:
passingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,TotPassCmp,KP,Prog
David de Gea,712.0,1.0,1.0
Paul Pogba,1728.0,51.0,274.0
Victor Lindelöf,1323.0,4.0,71.0
Luke Shaw,1657.0,33.0,238.0
Ashley Young,1380.0,42.0,259.0
Nemanja Matić,1579.0,14.0,152.0
Marcus Rashford,538.0,38.0,75.0
Chris Smalling,800.0,3.0,38.0
Romelu Lukaku,379.0,24.0,49.0
Jesse Lingard,668.0,17.0,49.0


### Possession Stats

In [12]:
def possessiontable(teamURL):
    '''
    input: teamURL(str)
    output: dataframe of the possession stats table with selected stats
    '''
    #URL html for shooting table read in beautifulsoup
    page = requests.get(teamURL).text
    soup = BeautifulSoup(page, 'lxml')
    placeholder = soup.select_one('#all_kitchen_sink_possession .placeholder')
    comment = next(elem for elem in placeholder.next_siblings
                   if isinstance(elem, Comment))
    possessionsoup = BeautifulSoup(comment, 'lxml')

    #finding shooting stats table
    possession_table = possessionsoup.find('table',
                                           id='stats_possession_ks_1889')

    #player names in list
    possessers = [
        header for header in possession_table.find_all('th', class_='left')
    ]
    possesser_names = [
        name.text for th in possessers for name in th.find_all('a')
    ]

    #player stats in list
    possession_row = [row for row in possession_table.find_all('tr')]

    possession_stats = {}
    for possesser in possession_row[2:]:
        items = possesser.find_all('td')
        possession_stats[possesser] = [i.text for i in items]
    possession_stats_list = [stats for stats in possession_stats.values()]

    #matching player name with stats
    all_possession = {}
    all_possession.update(zip(possesser_names, possession_stats_list))

    #creating dataframe for team's players and stats
    possession_df = pd.DataFrame(all_possession).T
    possession_df.columns = [
        'Nation', 'Pos', 'Age', '90s', 'Touches', 'Def Pen', 'Def 3rd',
        'Mid 3rd', 'Touches_Att3rd', 'Att Pen', 'Live', 'DribSucc', 'DribAtt',
        'DribSuccPerc', 'DribPl', 'Megs', 'Carries', 'TotDist', 'PrgDist',
        'RecTarg', 'Rec', 'RecTargPerc', 'Miscon', 'Dispos', 'Matches'
    ]

    #dropping unneded columns
    possession_df = possession_df.drop([
        'Nation', 'Pos', 'Age', '90s', 'Touches_Att3rd', 'Def Pen', 'Def 3rd',
        'Mid 3rd', 'Att Pen', 'Live', 'DribAtt', 'DribSuccPerc', 'DribSucc', 'Megs', 'Carries',
        'TotDist', 'PrgDist', 'RecTarg', 'RecTargPerc', 'Miscon', 'Dispos', 'Matches', 'Touches', 'Rec'
    ],
                                       axis=1)

    #data cleaning
    possession_df = possession_df.replace(r'^\s*$', np.nan, regex=True)
    possession_df.iloc[:, :] = possession_df.iloc[:, :].astype('float64')

    return possession_df

In [13]:
possessiontable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')

,DribPl
David de Gea,0.0
Paul Pogba,62.0
Victor Lindelöf,7.0
Luke Shaw,30.0
Ashley Young,23.0
Nemanja Matić,20.0
Marcus Rashford,52.0
Chris Smalling,1.0
Romelu Lukaku,27.0
Jesse Lingard,23.0


### Player Salaries

In [14]:
from unidecode import unidecode 

def salarytable(teamURL2):
    '''
    input: teamURL2 (str)
    output: dataframe of the salary table of selected team
    '''
    #URL html for salary table read in beautifulsoup
    page = requests.get(teamURL2).text
    soup = BeautifulSoup(page, 'lxml')

    #finding salary stats table
    salary_table = soup.find('table', attrs={'class': 'datatable rtable'})

    #player salaries in list
    player_row = [row for row in salary_table.find_all('tr')]

    player_salary = {}
    for player in player_row[1:]:
        items = player.find_all('td')
        player_name = items[0].text
        player_salary[player_name] = [i.text for i in items[1:]]

    #creating dataframe for team's players and salaries
    team_1819 = pd.DataFrame(player_salary).T
    team_1819.columns = [
        'Pos', 'Age', 'Blank', 'Annual_Salary(GBP)', 'Weekly_Salary'
    ]

    #dropping unneded columns
    team_1819 = team_1819.drop(['Pos', 'Age', 'Blank', 'Weekly_Salary'],
                               axis=1)

    #data cleaning
    team_1819 = team_1819.reset_index()
    team_1819['Annual_Salary(GBP)'] = team_1819['Annual_Salary(GBP)'].replace(
        '[^\w\s]', '', regex=True)
    team_1819['index'] = team_1819['index'].str.split(
        '\n', n=1).str[1].str.rstrip().apply(unidecode).str.lower()

    team_1819.set_index('index', inplace=True)

    return team_1819

In [15]:
salarytable('https://www.spotrac.com/epl/manchester-united-f.c/payroll/2018/')

,Annual_Salary(GBP)
index,
alexis sanchez,18200000
paul pogba,15080000
david de gea,10400000
romelu lukaku,9360000
juan mata,8320000
luke shaw,7800000
frederico de paula santos,6240000
victor lindelof,6240000
nemanja matic,6240000


### Final Team Stats

In [16]:
manutd_standard = standardtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_shooting = shootingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_passing = passingtable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_possession = possessiontable(
    'https://fbref.com/en/squads/19538871/2018-2019/Manchester-United-Stats')
manutd_salary = salarytable(
    'https://www.spotrac.com/epl/manchester-united-f.c/payroll/2018/')

In [17]:
def teamfinaltable(team_standard, team_shooting, team_passing, team_possession, team_salary):
    '''
    input: team_standard, team_shooting, team_passing, team_possession (dataframes)
    output: merge input into one final table (dataframe)
    '''
    teamfinal_1819 = team_standard.join(team_shooting)
    teamfinal_1819 = teamfinal_1819.join(team_passing)
    teamfinal_1819 = teamfinal_1819.join(team_possession)
    
    #standardizing all letters - getting rid of accents, etc.
    teamfinal_1819 = teamfinal_1819.reset_index()
    teamfinal_1819['index'] = teamfinal_1819['index'].apply(unidecode).str.lower()
    teamfinal_1819.set_index('index', inplace=True)
    
    teamfinal_1819 = teamfinal_1819.join(team_salary)

    return teamfinal_1819

In [18]:
#merging manutd tables into one table
manutd_final = teamfinaltable(manutd_standard, manutd_shooting, manutd_passing,
                              manutd_possession, manutd_salary)

In [19]:
manutd_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
david de gea,GK,27,38,38,1,0,0,38.0,0.00,0.00,712.0,1.0,1.0,0.0,10400000
paul pogba,MF,25,35,34,6,0.39,0.27,33.4,2.87,1.14,1728.0,51.0,274.0,62.0,15080000
victor lindelof,DF,24,30,29,1,0.03,0.03,28.9,0.24,0.03,1323.0,4.0,71.0,7.0,6240000
luke shaw,DF,23,29,29,11,0.03,0.14,28.8,0.69,0.21,1657.0,33.0,238.0,30.0,7800000
ashley young,DF,33,30,28,9,0.07,0.07,28.5,0.42,0.11,1380.0,42.0,259.0,23.0,6240000
nemanja matic,MF,29,28,28,7,0.04,0,27.1,0.44,0.11,1579.0,14.0,152.0,20.0,6240000
marcus rashford,FW,20,33,26,3,0.39,0.23,25.9,3.20,1.54,538.0,38.0,75.0,52.0,2340000
chris smalling,DF,28,24,24,1,0.04,0,23.6,0.51,0.17,800.0,3.0,38.0,1.0,4160000
romelu lukaku,FW,25,32,22,4,0.51,0,23.7,2.49,1.39,379.0,24.0,49.0,27.0,9360000


## Manchester City

### Final Team Stats

In [20]:
mancity_standard = standardtable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_shooting = shootingtable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_passing = passingtable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_possession = possessiontable(
    'https://fbref.com/en/squads/b8fd03ef/2018-2019/Manchester-City-Stats')
mancity_salary = salarytable(
    'https://www.spotrac.com/epl/manchester-city-f.c/payroll/2018/')

In [21]:
#merging mancity tables into one table
mancity_final = teamfinaltable(mancity_standard, mancity_shooting,
                               mancity_passing, mancity_possession, mancity_salary)

In [22]:
mancity_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
ederson,GK,24,38,38,2,0,0.03,38.0,0.00,0.00,918.0,3.0,3.0,1.0,NaN
aymeric laporte,DF,24,35,34,3,0.09,0.09,34.0,0.79,0.21,2793.0,14.0,251.0,3.0,3380000
bernardo silva,"MF,FW",23,36,31,3,0.22,0.22,31.7,2.02,0.69,1513.0,70.0,168.0,55.0,6240000
raheem sterling,FW,23,34,31,3,0.55,0.29,30.8,2.47,1.20,1099.0,58.0,100.0,91.0,9360000
sergio aguero,FW,30,33,31,4,0.77,0.29,27.3,4.32,1.50,665.0,34.0,68.0,32.0,13000000
kyle walker,DF,28,33,30,3,0.03,0.03,30.9,0.65,0.10,2337.0,24.0,189.0,25.0,5200000
david silva,MF,32,33,28,2,0.22,0.3,26.7,1.87,0.67,1783.0,73.0,204.0,31.0,11440000
fernandinho,MF,33,29,27,5,0.04,0.11,26.4,1.17,0.27,1804.0,24.0,218.0,12.0,NaN
ilkay gundogan,MF,27,31,23,3,0.25,0.13,23.7,1.81,0.46,1837.0,43.0,201.0,18.0,6240000


## Liverpool

### Final Team Stats

In [23]:
liverpool_standard = standardtable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_shooting = shootingtable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_passing = passingtable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_possession = possessiontable(
    'https://fbref.com/en/squads/822bd0ba/2018-2019/Liverpool-Stats')
liverpool_salary = salarytable(
    'https://www.spotrac.com/epl/liverpool-f.c/payroll/2018/')

In [24]:
#merging liverpool tables into one table
liverpool_final = teamfinaltable(liverpool_standard, liverpool_shooting,
                                 liverpool_passing, liverpool_possession, liverpool_salary)

In [25]:
liverpool_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
alisson,GK,25,38,38,1,0,0,38.0,0.00,0.00,1039.0,0.0,0.0,1.0,NaN
virgil van dijk,DF,27,38,38,1,0.11,0.05,37.6,0.85,0.21,2709.0,6.0,157.0,8.0,9360000
mohamed salah,FW,26,38,37,1,0.61,0.22,36.1,3.74,1.66,864.0,68.0,158.0,86.0,10400000
andrew robertson,DF,24,36,36,4,0,0.31,35.7,0.39,0.14,2283.0,52.0,321.0,32.0,2600000
sadio mane,FW,26,36,35,2,0.64,0.03,34.2,2.57,1.23,932.0,43.0,111.0,67.0,5200000
georginio wijnaldum,MF,27,35,32,3,0.1,0,30.3,0.99,0.26,1620.0,21.0,131.0,36.0,3900000
roberto firmino,"FW,MF",26,34,31,0,0.41,0.21,29.0,2.52,1.07,1056.0,42.0,158.0,60.0,9360000
trent alexander-arnold,DF,19,29,27,3,0.04,0.44,27.4,1.06,0.33,1644.0,48.0,322.0,23.0,NaN
fabinho,MF,24,28,21,6,0.04,0.09,22.4,0.76,0.18,1403.0,16.0,135.0,16.0,NaN


## Chelsea

### Final Team Stats

In [26]:
chelsea_standard = standardtable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_shooting = shootingtable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_passing = passingtable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_possession = possessiontable(
    'https://fbref.com/en/squads/cff3d9bb/2018-2019/Chelsea-Stats')
chelsea_salary = salarytable(
    'https://www.spotrac.com/epl/chelsea-f.c/payroll/2018/')

In [27]:
#merging chelsea tables into one table
chelsea_final = teamfinaltable(chelsea_standard, chelsea_shooting,
                               chelsea_passing, chelsea_possession, chelsea_salary)

In [28]:
chelsea_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
cesar azpilicueta,DF,28,38,38,4,0.03,0.13,37.8,0.45,0.19,2471.0,25.0,231.0,17.0,4680000
jorginho,MF,26,37,37,8,0.06,0,35.1,0.51,0.17,2703.0,29.0,317.0,16.0,NaN
kepa arrizabalaga,GK,23,36,36,2,0,0,36.0,0.00,0.00,974.0,0.0,1.0,1.0,7800000
david luiz,DF,31,36,36,3,0.08,0.06,36.0,0.92,0.22,2293.0,16.0,232.0,8.0,6240000
n'golo kante,MF,27,36,36,3,0.12,0.12,34.4,0.84,0.17,1805.0,47.0,150.0,36.0,7500000
antonio rudiger,DF,25,33,33,7,0.03,0,31.9,0.88,0.22,2310.0,5.0,149.0,5.0,6000000
eden hazard,FW,27,37,32,2,0.49,0.46,32.5,2.77,1.20,1595.0,93.0,269.0,143.0,11700000
marcos alonso,DF,27,31,31,6,0.07,0.13,30.7,1.40,0.23,1838.0,31.0,175.0,25.0,5200000
willian,FW,29,32,26,2,0.13,0.26,23.3,2.66,0.94,1100.0,82.0,186.0,41.0,NaN


## Tottenham

### Final Team Stats

In [29]:
tottenham_standard = standardtable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_shooting = shootingtable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_passing = passingtable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_possession = possessiontable(
    'https://fbref.com/en/squads/361ca564/2018-2019/Tottenham-Hotspur-Stats')
tottenham_salary = salarytable(
    'https://www.spotrac.com/epl/tottenham-hotspur-f.c/payroll/2018/')

In [30]:
#merging tottenham tables into one table
tottenham_final = teamfinaltable(tottenham_standard, tottenham_shooting,
                                 tottenham_passing, tottenham_possession, tottenham_salary)

In [31]:
tottenham_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
hugo lloris,GK,31,33,33,0,0,0,33.0,0.00,0.00,890.0,1.0,0.0,0.0,5200000
toby alderweireld,DF,29,34,33,3,0,0,32.5,0.52,0.18,2067.0,7.0,133.0,2.0,NaN
christian eriksen,"MF,FW",26,35,30,3,0.26,0.39,30.8,2.73,1.01,1540.0,75.0,228.0,23.0,3900000
harry kane,FW,25,28,27,5,0.63,0.15,26.9,3.60,1.56,423.0,29.0,81.0,45.0,10400000
moussa sissoko,MF,28,29,27,2,0,0.12,25.9,0.58,0.12,1205.0,23.0,102.0,49.0,4160000
kieran trippier,DF,27,27,26,2,0.04,0.12,25.3,0.32,0.12,1515.0,48.0,262.0,10.0,3380000
lucas moura,"FW,MF",25,32,25,3,0.42,0,23.6,1.91,0.85,561.0,27.0,61.0,46.0,4160000
son heung-min,"FW,MF",26,31,23,2,0.53,0.26,22.7,3.31,1.24,646.0,31.0,71.0,62.0,NaN
davinson sanchez,DF,22,23,22,2,0.05,0.05,21.6,0.56,0.14,1289.0,1.0,52.0,0.0,3380000


## Arsenal

### Final Team Stats

In [32]:
arsenal_standard = standardtable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_shooting = shootingtable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_passing = passingtable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_possession = possessiontable(
    'https://fbref.com/en/squads/18bb7c10/2018-2019/Arsenal-Stats')
arsenal_salary = salarytable(
    'https://www.spotrac.com/epl/arsenal-f.c/payroll/2018/')

In [33]:
#merging arsenal tables into one table
arsenal_final = teamfinaltable(arsenal_standard, arsenal_shooting,
                               arsenal_passing, arsenal_possession, arsenal_salary)

In [34]:
arsenal_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
bernd leno,GK,26,32,31,0,0,0,31.5,0.00,0.00,779.0,0.0,2.0,0.0,5200000
shkodran mustafi,DF,26,31,31,9,0.07,0,29.0,0.65,0.31,1438.0,5.0,105.0,12.0,4680000
pierre-emerick aubameyang,FW,29,36,30,0,0.73,0.17,30.3,2.94,1.12,526.0,33.0,68.0,32.0,10400000
granit xhaka,"MF,DF",25,29,29,10,0.14,0.07,27.8,1.08,0.40,1955.0,38.0,272.0,16.0,5200000
alexandre lacazette,FW,27,35,27,2,0.47,0.29,27.8,2.91,1.01,579.0,29.0,79.0,53.0,9467273
sokratis papastathopoulos,DF,30,25,25,12,0.04,0.08,24.4,0.41,0.04,1215.0,4.0,40.0,8.0,4784000
lucas torreira,MF,22,34,24,7,0.08,0.08,26.5,0.91,0.23,1420.0,16.0,137.0,23.0,3900000
matteo guendouzi,MF,19,33,23,9,0,0,23.9,0.63,0.13,1412.0,19.0,146.0,33.0,2080000
alex iwobi,"FW,MF",22,35,22,0,0.14,0.27,21.9,1.60,0.64,774.0,39.0,130.0,58.0,2600000


## Wolves

### Final Team Stats

In [35]:
wolves_standard = standardtable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_shooting = shootingtable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_passing = passingtable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_possession = possessiontable(
    'https://fbref.com/en/squads/8cec06e1/2018-2019/Wolverhampton-Wanderers-Stats'
)
wolves_salary = salarytable(
    'https://www.spotrac.com/epl/wolverhampton-wanderers-f.c/payroll/2018/'
)

In [36]:
#merging wolves tables into one table
wolves_final = teamfinaltable(wolves_standard, wolves_shooting, wolves_passing,
                              wolves_possession, wolves_salary)

In [37]:
wolves_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
conor coady,DF,25,38,38,5,0,0,38.0,0.00,0.00,1412.0,0.0,61.0,2.0,780000
rui patricio,GK,30,37,37,1,0,0,37.0,0.00,0.00,493.0,0.0,0.0,0.0,5200000
willy boly,DF,27,36,36,2,0.11,0,35.2,0.63,0.17,1392.0,10.0,125.0,17.0,2080000
raul jimenez,FW,27,38,36,4,0.38,0.2,34.6,3.27,0.93,789.0,42.0,81.0,37.0,2132000
matt doherty,"DF,MF",26,38,35,5,0.11,0.14,34.9,1.23,0.49,1333.0,41.0,294.0,30.0,520000
joao moutinho,MF,31,38,35,4,0.03,0.24,33.7,0.53,0.09,1660.0,80.0,263.0,15.0,NaN
ryan bennett,DF,28,34,34,12,0.03,0,34.0,0.44,0.09,1095.0,5.0,95.0,1.0,1040000
ruben neves,MF,21,35,34,8,0.12,0.09,33.4,2.13,0.45,1569.0,19.0,176.0,14.0,2600000
jonny castro,"DF,MF",24,33,32,6,0.03,0.03,30.8,0.84,0.23,1124.0,25.0,107.0,27.0,1907000


## Everton

### Final Team Stats

In [38]:
everton_standard = standardtable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_shooting = shootingtable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_passing = passingtable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_possession = possessiontable(
    'https://fbref.com/en/squads/d3fd31cc/2018-2019/Everton-Stats')
everton_salary = salarytable(
    'https://www.spotrac.com/epl/everton-f.c/payroll/2018/')

In [39]:
#merging everton tables into one table
everton_final = teamfinaltable(everton_standard, everton_shooting,
                               everton_passing, everton_possession, everton_salary)

In [40]:
everton_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
jordan pickford,GK,24,38,38,1,0,0,38.0,0.00,0.00,748.0,1.0,1.0,0.0,5200000
gylfi sigurdsson,MF,28,38,36,3,0.38,0.17,34.6,2.34,0.84,830.0,72.0,190.0,42.0,5200000
michael keane,DF,25,33,33,2,0.03,0.06,33.0,0.88,0.27,1285.0,8.0,108.0,7.0,3120000
lucas digne,DF,25,35,33,5,0.12,0.12,32.9,1.03,0.30,1460.0,70.0,293.0,16.0,4680000
idrissa gana gueye,MF,28,33,32,6,0,0.06,31.4,0.64,0.10,1478.0,19.0,171.0,23.0,NaN
richarlison,"FW,MF",21,35,32,5,0.44,0.03,29.6,2.94,0.91,546.0,17.0,76.0,50.0,NaN
kurt zouma,DF,23,32,29,4,0.07,0.07,29.0,0.59,0.14,1218.0,6.0,87.0,5.0,4000000
seamus coleman,DF,29,29,29,1,0.07,0.07,28.4,0.46,0.18,1218.0,22.0,188.0,26.0,3640000
bernard,"FW,MF",25,34,25,5,0.04,0.13,23.5,1.02,0.21,633.0,40.0,91.0,48.0,NaN


## Leicester City

### Final Team Stats

In [41]:
leicester_standard = standardtable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_shooting = shootingtable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_passing = passingtable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_possession = possessiontable(
    'https://fbref.com/en/squads/a2d435b3/2018-2019/Leicester-City-Stats')
leicester_salary = salarytable(
    'https://www.spotrac.com/epl/leicester-city/payroll/2018/')

In [42]:
#merging leicester tables into one table
leicester_final = teamfinaltable(leicester_standard, leicester_shooting,
                                 leicester_passing, leicester_possession, leicester_salary)

In [43]:
leicester_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
kasper schmeichel,GK,31,38,38,3,0,0,38.0,0.03,0.00,825.0,1.0,3.0,2.0,6760000
wilfred ndidi,MF,21,38,37,8,0.05,0,36.6,1.53,0.25,1634.0,22.0,153.0,45.0,NaN
ben chilwell,DF,21,36,36,4,0,0.11,36.0,0.56,0.28,1575.0,40.0,211.0,44.0,3380000
ricardo pereira,"DF,FW",24,35,35,7,0.06,0.17,34.7,0.75,0.26,1495.0,31.0,224.0,72.0,3640000
james maddison,"MF,FW",21,36,35,4,0.22,0.22,31.5,2.61,0.86,1243.0,96.0,237.0,46.0,2860000
harry maguire,DF,25,31,31,6,0.1,0,28.8,1.01,0.38,1496.0,9.0,166.0,17.0,4680000
jamie vardy,FW,31,34,30,3,0.59,0.13,30.3,2.48,1.06,286.0,29.0,38.0,21.0,7280000
nampalys mendy,MF,26,31,23,5,0,0,23.2,0.13,0.00,969.0,6.0,80.0,21.0,2600000
demarai gray,"FW,MF",22,34,23,2,0.18,0.05,22.1,2.09,0.86,523.0,11.0,46.0,46.0,2860000


## West Ham

### Final Team Stats

In [44]:
westham_standard = standardtable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_shooting = shootingtable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_passing = passingtable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_possession = possessiontable(
    'https://fbref.com/en/squads/7c21e445/2018-2019/West-Ham-United-Stats')
westham_salary = salarytable(
    'https://www.spotrac.com/epl/west-ham-united-f.c/payroll/2018/')

In [45]:
#merging westham tables into one table
westham_final = teamfinaltable(westham_standard, westham_shooting,
                               westham_passing, westham_possession, westham_salary)

In [46]:
westham_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
lukasz fabianski,GK,33,38,38,0,0,0,38.0,0.00,0.00,651.0,3.0,0.0,1.0,3380000
felipe anderson,"FW,MF",25,36,36,3,0.27,0.12,33.8,1.72,0.71,1328.0,60.0,223.0,89.0,4420000
declan rice,MF,19,34,34,4,0.06,0,33.4,0.72,0.15,1364.0,9.0,85.0,25.0,156000
issa diop,DF,21,33,33,4,0.03,0,33.0,0.42,0.12,973.0,2.0,64.0,5.0,3484000
mark noble,MF,31,31,29,6,0.2,0.2,25.5,0.51,0.20,1293.0,21.0,163.0,16.0,2600000
robert snodgrass,"MF,FW",30,33,25,10,0.08,0.2,24.7,1.26,0.32,875.0,61.0,138.0,37.0,2080000
marko arnautovic,FW,29,28,24,3,0.45,0.18,22.1,2.80,1.36,496.0,33.0,46.0,29.0,5200000
pablo zabaleta,DF,33,26,23,4,0,0.04,23.2,0.30,0.04,863.0,7.0,145.0,10.0,3900000
fabian balbuena,DF,26,23,23,2,0.04,0,22.4,0.76,0.09,655.0,3.0,42.0,2.0,2080000


## Watford

### Final Team Stats

In [47]:
watford_standard = standardtable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_shooting = shootingtable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_passing = passingtable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_possession = possessiontable(
    'https://fbref.com/en/squads/2abfe087/2018-2019/Watford-Stats')
watford_salary = salarytable(
    'https://www.spotrac.com/epl/watford/payroll/2018/')

In [48]:
#merging watford tables into one table
watford_final = teamfinaltable(watford_standard, watford_shooting,
                               watford_passing, watford_possession, watford_salary)

In [49]:
watford_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
ben foster,GK,35,38,38,1,0,0,38.0,0.03,0.00,600.0,7.0,2.0,0.0,1560000
craig cathcart,DF,29,36,35,2,0.09,0,34.4,0.47,0.17,1046.0,1.0,64.0,3.0,1200000
abdoulaye doucoure,MF,25,35,34,7,0.15,0.18,34.0,1.53,0.47,1640.0,37.0,165.0,44.0,3640000
etienne capoue,MF,30,33,33,14,0.03,0.09,32.7,0.52,0.09,1482.0,9.0,164.0,32.0,2080000
roberto pereyra,"FW,MF",27,33,33,3,0.19,0.03,31.3,1.79,0.67,967.0,38.0,125.0,54.0,2340000
will hughes,"MF,FW",23,32,31,5,0.07,0.15,26.9,1.15,0.30,773.0,28.0,108.0,23.0,1560000
troy deeney,FW,30,32,28,4,0.32,0.18,28.3,2.16,0.99,522.0,40.0,109.0,10.0,3380000
jose holebas,DF,34,28,27,12,0.11,0.23,26.6,0.64,0.23,819.0,36.0,175.0,25.0,1560000
gerard deulofeu,FW,24,30,26,3,0.44,0.22,23.0,2.70,1.00,580.0,30.0,85.0,46.0,3120000


## Crystal Palace

### Final Team Stats

In [50]:
crystalpalace_standard = standardtable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_shooting = shootingtable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_passing = passingtable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_possession = possessiontable(
    'https://fbref.com/en/squads/47c64c55/2018-2019/Crystal-Palace-Stats')
crystalpalace_salary = salarytable(
    'https://www.spotrac.com/epl/crystal-palace/payroll/2018/')

In [51]:
#merging crystalpalace tables into one table
crystalpalace_final = teamfinaltable(crystalpalace_standard,
                                     crystalpalace_shooting,
                                     crystalpalace_passing,
                                     crystalpalace_possession, crystalpalace_salary)

In [52]:
crystalpalace_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
luka milivojevic,MF,27,38,38,10,0.32,0.05,38.0,1.24,0.34,1629.0,59.0,248.0,20.0,2860000
patrick van aanholt,DF,27,36,36,3,0.08,0.06,35.5,1.24,0.34,1587.0,34.0,292.0,28.0,2860000
james mcarthur,MF,30,38,36,7,0.09,0.18,34.0,1.00,0.24,1151.0,30.0,101.0,30.0,2860000
aaron wan-bissaka,DF,20,35,35,4,0,0.09,34.8,0.26,0.00,1156.0,14.0,164.0,78.0,780000
wilfried zaha,FW,25,34,34,9,0.3,0.15,33.7,2.34,0.89,839.0,54.0,147.0,155.0,6760000
andros townsend,"FW,MF",27,38,34,1,0.18,0.12,33.4,1.85,0.51,895.0,64.0,134.0,79.0,2860000
james tomkins,DF,29,29,29,5,0.04,0.04,28.4,0.81,0.25,915.0,6.0,51.0,1.0,2600000
mamadou sakho,DF,28,27,27,2,0,0,26.8,0.19,0.00,1135.0,7.0,100.0,11.0,5200000
cheikhou kouyate,MF,28,31,21,3,0,0.09,21.1,1.00,0.38,639.0,15.0,70.0,20.0,3380000


## Newcastle United

### Final Team Stats

In [53]:
newcastle_standard = standardtable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_shooting = shootingtable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_passing = passingtable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_possession = possessiontable(
    'https://fbref.com/en/squads/b2b47a98/2018-2019/Newcastle-United-Stats')
newcastle_salary = salarytable(
    'https://www.spotrac.com/epl/newcastle-united-f.c/payroll/2018/')

In [54]:
#merging newcastle tables into one table
newcastle_final = teamfinaltable(newcastle_standard, newcastle_shooting,
                                 newcastle_passing, newcastle_possession, newcastle_salary)

In [55]:
newcastle_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
martin dubravka,GK,29,38,38,1,0,0,38.0,0.00,0.00,686.0,0.0,0.0,1.0,1976000
matt ritchie,"DF,MF",28,36,35,9,0.06,0.25,32.5,1.20,0.22,1036.0,53.0,216.0,17.0,2340000
ayoze perez,"FW,MF",25,37,34,2,0.37,0.06,32.4,1.64,0.80,609.0,38.0,68.0,67.0,2340000
jamaal lascelles,DF,24,32,32,4,0,0.03,30.5,0.33,0.03,836.0,2.0,53.0,3.0,2080000
salomon rondon,FW,28,32,30,1,0.38,0.24,28.9,3.08,1.00,538.0,30.0,55.0,24.0,NaN
deandre yedlin,DF,25,29,28,6,0.04,0.07,27.5,0.40,0.07,784.0,29.0,156.0,10.0,1560000
mohamed diame,MF,31,29,24,3,0,0,24.1,0.87,0.12,571.0,1.0,45.0,27.0,2340000
fabian schar,DF,26,24,22,12,0.18,0.05,22.2,1.04,0.32,699.0,11.0,77.0,13.0,2080000
paul dummett,DF,26,26,21,1,0,0,22.1,0.36,0.05,639.0,5.0,104.0,3.0,1820000


## Bournemouth

### Final Team Stats

In [56]:
bournemouth_standard = standardtable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_shooting = shootingtable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_passing = passingtable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_possession = possessiontable(
    'https://fbref.com/en/squads/4ba7cbea/2018-2019/Bournemouth-Stats')
bournemouth_salary = salarytable(
    'https://www.spotrac.com/epl/afc-bournemouth/payroll/2018/')

In [57]:
#merging bournemouth tables into one table
bournemouth_final = teamfinaltable(bournemouth_standard, bournemouth_shooting,
                                   bournemouth_passing, bournemouth_possession, bournemouth_salary)

In [58]:
bournemouth_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
nathan ake,DF,23,38,38,3,0.11,0,37.9,0.58,0.21,1469.0,5.0,91.0,3.0,2080000
ryan fraser,"MF,FW",24,38,35,2,0.2,0.4,35.2,1.42,0.68,1019.0,87.0,225.0,25.0,1404000
joshua king,FW,26,35,34,3,0.36,0.09,32.9,1.95,0.55,607.0,37.0,94.0,79.0,2340000
steve cook,DF,27,31,31,3,0.03,0.03,31.0,0.71,0.26,1222.0,12.0,117.0,4.0,2340000
callum wilson,FW,26,30,29,3,0.5,0.32,28.1,2.24,0.96,319.0,28.0,45.0,27.0,2080000
jefferson lerma,MF,23,30,29,12,0.07,0,27.8,1.19,0.29,1320.0,15.0,117.0,28.0,2600000
david brooks,"MF,FW",21,30,29,3,0.28,0.2,25.1,1.63,0.84,649.0,29.0,110.0,49.0,1040000
adam smith,DF,27,25,25,5,0.04,0.04,23.0,0.35,0.04,860.0,10.0,116.0,19.0,1820000
asmir begovic,GK,31,24,24,0,0,0,24.0,0.00,0.00,431.0,0.0,1.0,0.0,3640000


## Burnley

### Final Team Stats

In [59]:
burnley_standard = standardtable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_shooting = shootingtable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_passing = passingtable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_possession = possessiontable(
    'https://fbref.com/en/squads/943e8050/2018-2019/Burnley-Stats')
burnley_salary = salarytable(
    'https://www.spotrac.com/epl/burnley-f.c/payroll/2018/')

In [60]:
#merging burnley tables into one table
burnley_final = teamfinaltable(burnley_standard, burnley_shooting,
                               burnley_passing, burnley_possession, burnley_salary)

In [61]:
burnley_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
ben mee,DF,28,38,38,9,0,0.05,38.0,0.61,0.16,1119.0,9.0,99.0,9.0,2860000
jack cork,MF,29,37,37,6,0.03,0.05,36.8,0.41,0.05,1081.0,17.0,95.0,29.0,1976000
charlie taylor,DF,24,38,35,2,0,0.03,36.0,0.17,0.00,1224.0,12.0,163.0,43.0,1300000
james tarkowski,DF,25,35,35,8,0.09,0.03,34.3,0.70,0.23,790.0,6.0,102.0,9.0,2600000
ashley westwood,MF,28,34,31,5,0.06,0.22,31.4,0.70,0.22,1111.0,43.0,200.0,11.0,1612000
chris wood,FW,26,38,29,2,0.35,0.07,28.8,2.02,0.80,406.0,15.0,26.0,15.0,2600000
ashley barnes,FW,28,37,26,8,0.45,0.08,26.5,2.30,1.02,346.0,20.0,45.0,12.0,2080000
jeff hendrick,"MF,FW",26,32,25,4,0.13,0,22.9,0.83,0.26,511.0,15.0,59.0,18.0,1820000
johann berg gudmundsson,"MF,FW",27,29,19,2,0.15,0.31,19.5,1.75,0.57,407.0,40.0,87.0,19.0,1560000


## Southampton

### Final Team Stats

In [62]:
southampton_standard = standardtable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_shooting = shootingtable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_passing = passingtable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_possession = possessiontable(
    'https://fbref.com/en/squads/33c895d4/2018-2019/Southampton-Stats')
southampton_salary = salarytable(
    'https://www.spotrac.com/epl/southampton-f.c/payroll/2018/')

In [63]:
#merging southampton tables into one table
southampton_final = teamfinaltable(southampton_standard, southampton_shooting,
                                   southampton_passing, southampton_possession, southampton_salary)

In [64]:
southampton_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
nathan redmond,"FW,MF",24,38,36,3,0.17,0.11,36.3,2.04,0.61,877.0,55.0,111.0,116.0,3120000
pierre hojbjerg,MF,22,31,31,8,0.13,0.1,30.7,1.73,0.62,1495.0,27.0,165.0,31.0,NaN
alex mccarthy,GK,28,25,25,1,0,0,25.0,0.00,0.00,318.0,0.0,0.0,1.0,2600000
oriol romeu,MF,26,31,25,11,0.04,0,23.7,0.42,0.17,898.0,7.0,130.0,12.0,3380000
ryan bertrand,DF,28,24,24,8,0.04,0,24.0,0.62,0.08,795.0,23.0,161.0,11.0,3640000
jan bednarek,DF,22,25,24,7,0,0.04,23.9,0.25,0.04,591.0,7.0,71.0,2.0,1040000
jannik vestergaard,DF,25,23,23,2,0,0,22.5,0.58,0.04,671.0,6.0,73.0,12.0,3120000
danny ings,FW,26,24,23,1,0.38,0.16,18.3,2.83,1.04,287.0,25.0,42.0,37.0,3900000
james ward-prowse,"MF,DF",23,26,21,4,0.32,0,21.6,1.39,0.65,622.0,44.0,88.0,7.0,2340000


## Brighton

### Final Team Stats

In [65]:
brighton_standard = standardtable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_shooting = shootingtable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_passing = passingtable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_possession = possessiontable(
    'https://fbref.com/en/squads/d07537b9/2018-2019/Brighton-and-Hove-Albion-Stats'
)
brighton_salary = salarytable(
    'https://www.spotrac.com/epl/brighton-hove-albion/payroll/2018/'
)

In [66]:
#merging brighton tables into one table
brighton_final = teamfinaltable(brighton_standard, brighton_shooting,
                                brighton_passing, brighton_possession,
                                brighton_salary)

In [67]:
brighton_final

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
lewis dunk,DF,26,36,36,6,0.06,0.03,35.0,0.63,0.11,1240.0,9.0,50.0,1.0,2340000
shane duffy,DF,26,35,35,4,0.15,0.03,34.3,1.14,0.41,990.0,6.0,33.0,5.0,2080000
mathew ryan,GK,26,34,34,2,0,0,34.0,0.00,0.00,551.0,0.0,0.0,0.0,1820000
davy propper,MF,26,30,30,2,0.04,0.04,28.3,0.53,0.07,980.0,19.0,130.0,40.0,2340000
glenn murray,FW,34,38,30,5,0.47,0.04,27.9,1.83,0.57,385.0,11.0,36.0,16.0,1560000
solly march,"FW,MF",24,35,30,1,0.04,0.18,27.4,1.53,0.33,597.0,38.0,86.0,48.0,1560000
dale stephens,MF,29,30,29,6,0.04,0.04,27.6,0.36,0.07,1145.0,21.0,126.0,11.0,1560000
martin montoya,DF,27,25,24,4,0,0.04,24.1,0.33,0.04,895.0,19.0,148.0,15.0,2600000
pascal gross,MF,27,25,24,1,0.14,0.14,20.7,1.01,0.34,654.0,53.0,118.0,10.0,2600000


# Final Merge and Clean on 2018-2019 Stats

## Merging Teams Into One Table

In [68]:
def totalmerge():
    '''
    Merging all team_final tables (dataframe) into one table (dataframe)
    '''
    teams = [
        manutd_final,
        mancity_final,
        liverpool_final,
        chelsea_final,
        tottenham_final,
        arsenal_final,
        wolves_final,
        everton_final,
        leicester_final,
        westham_final,
        watford_final,
        crystalpalace_final,
        newcastle_final,
        bournemouth_final,
        burnley_final,
        southampton_final,
        brighton_final,
    ]
    total_1819 = pd.concat(teams)

    return total_1819

In [69]:
total_1819 = totalmerge()

## Final Data Cleaning

In [70]:
total_1819

,Pos,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP)
index,,,,,,,,,,,,,,,
david de gea,GK,27,38,38,1,0,0,38.0,0.00,0.00,712.0,1.0,1.0,0.0,10400000
paul pogba,MF,25,35,34,6,0.39,0.27,33.4,2.87,1.14,1728.0,51.0,274.0,62.0,15080000
victor lindelof,DF,24,30,29,1,0.03,0.03,28.9,0.24,0.03,1323.0,4.0,71.0,7.0,6240000
luke shaw,DF,23,29,29,11,0.03,0.14,28.8,0.69,0.21,1657.0,33.0,238.0,30.0,7800000
ashley young,DF,33,30,28,9,0.07,0.07,28.5,0.42,0.11,1380.0,42.0,259.0,23.0,6240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
jayson molumby,MF,18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max sanders,MF,19,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jason steele,GK,27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,833333


In [71]:
#dropping players that never played in a premier league match in the 2018-2019 season
total_1819 = total_1819[~(total_1819['MP'] == 0)]

#changing all number columns to float
total_1819.iloc[:, 1:] = total_1819.iloc[:, 1:].apply(pd.to_numeric,
                                                      errors='coerce')

#changing NaN to 0
total_1819 = total_1819.fillna(0)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [72]:
def standardposition(Pos):
    '''
    input: Pos (series) column for player position
    output: standardizes a player's position to his primary position if they have multiple
    '''
    for elem in Pos:
        return elem.split(',')[0]

In [73]:
total_1819['Pos'] = total_1819['Pos'].apply(standardposition)

In [74]:
#checking for players that played on two different premier league teams in the same year due to being loaned out
premleague = total_1819.reset_index()
premleague['index'].value_counts(ascending=False)

nathaniel clyne      2
hector bellerin      1
anthony martial      1
leroy sane           1
manolo gabbiadini    1
                    ..
shkodran mustafi     1
marcus rashford      1
martin dubravka      1
will hughes          1
fred                 1
Name: index, Length: 423, dtype: int64

In [75]:
#separating out player position for future merge
playerposition = premleague.iloc[:, 0:2]
playerposition.drop_duplicates('index', inplace=True)

#merging stats for loaned out players
premleague = premleague.groupby('index').sum().reset_index()

#dropping duplicate player rows
premleague = premleague.drop_duplicates(subset="index")

#adding player position back in
premleague = premleague.merge(playerposition,
                              left_on='index',
                              right_on='index')

In [76]:
#example loaned out player
premleague[premleague['index']=='nathaniel clyne']

,index,Age,MP,Starts,CrdY,P90Gls,P90Ast,90s,Sh/90,SoT/90,TotPassCmp,KP,Prog,DribPl,Annual_Salary(GBP),Pos
311,nathaniel clyne,54.0,18.0,14.0,2.0,0.0,0.08,13.9,2.59,0.0,522.0,5.0,86.0,6.0,3640000.0,D


# Exporting 2018-2019 Stats

In [77]:
premleague.to_csv('premleague_salary_stats.csv')